In [ ]:
import tensorflow as tf
from keras.applications import VGG16

##### Loading pre-trained VGG16 model without including the top three fully connected layers

In [ ]:
base_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(64,64,3)
)

In [ ]:
base_model.get_weights()

In [ ]:
x = base_model.output # Output of VGG16 is tensor(4-D Tensor) of shape (None, 2, 2, 512). This shape depends on the shape of input image and filters used.
x

In [ ]:
base_model.input

In [ ]:
# Flatten the output of base model
x = tf.keras.layers.Flatten()(x)
fc1 = tf.keras.layers.Dense(units=256, activation='relu')(x)
fc2 = tf.keras.layers.Dense(units=256, activation='relu')(fc1)
fc3 = tf.keras.layers.Dense(units=256, activation='relu')(fc2)
fc4 = tf.keras.layers.Dense(units=256, activation='relu')(fc3)

In [ ]:
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(fc4)

In [ ]:
model = tf.keras.Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
base_model.trainable = False


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Data Augmentation - Data augmentation is a set of techniques to artificially increase the amount 
# of data by generating new data points from existing data
train_datagen = ImageDataGenerator(
    rescale=1./255,     #feature scaling--->putting all values between 0 and 1
    shear_range=0.2,    #Shear angle in the counter-clockwise direction in degrees
    zoom_range=0.2,
    horizontal_flip=True    #Radomly flip inputs horizontally
)    

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory('C:/Users/User/MachineLearning/TransferLearning/training_set', target_size=(64,64), batch_size=32, class_mode='binary')

In [ ]:
test_set = test_datagen.flow_from_directory('C:/Users/User/MachineLearning/TransferLearning/test_set', target_size=(64,64), batch_size=32, class_mode='binary')

In [ ]:
train_set.class_indices

In [ ]:
history = model.fit(train_set, epochs=20, validation_data=test_set)

In [ ]:
model.get_config()

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'],label='train_loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()


In [ ]:
plt.plot(history.history['accuracy'],label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
model.save('C:/Users/User/MachineLearning/TransferLearning/vgg_16_2.h5')

In [ ]:
import numpy as np
from keras.utils import load_img, img_to_array

In [ ]:
test_image = load_img('C:/Users/User/MachineLearning/TransferLearning/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))

In [ ]:
test_image2 = load_img('C:/Users/User/MachineLearning/TransferLearning/single_prediction/cat_or_dog_2.jpg', target_size=(64,64))

In [ ]:
test_image = img_to_array(test_image)
test_image2 = img_to_array(test_image2)

In [ ]:
test_image = np.expand_dims(test_image, axis=0)
test_image2 = np.expand_dims(test_image2, axis=0)

In [ ]:
result = model.predict(test_image)

In [ ]:
result2 = model.predict(test_image2)

In [ ]:
result

In [ ]:
result2

In [ ]:
train_set.class_indices

### The model predicts the output output. The second image is a cat and it identifies it as dog.